In [14]:
import pandas as pd
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


data = pd.read_csv('/Users/Boaz/Documents/DataMining/prepro_data_RF.csv')
data['mood'] = round(data['mood'])
data['circumplex.arousal'] = round(data['circumplex.arousal'])
data['circumplex.valence'] = round(data['circumplex.valence'])
data['target_mood_plus1'] = round(data['target_mood_plus1'])

In [15]:
data

,id,time,mood,circumplex.arousal,circumplex.valence,activity,screen,sms,appCat.builtin,appCat.communication,...,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,target_mood_plus1,season,summer,spring,winter
0,AS14.01,2014-02-26,6,-0.0,1.0,0.000000,0.000000,2.0,0.000,0.000,...,0.0000,0.000,0.0,0.000,0.000,6.0,winter,0,0,1
1,AS14.01,2014-03-21,6,0.0,0.0,3.083152,9100.601001,0.0,1255.653,5903.482,...,3945.8130,915.445,0.0,598.754,0.000,6.0,spring,0,1,0
2,AS14.01,2014-03-22,6,1.0,0.0,3.790084,6142.161000,1.0,480.124,4962.918,...,439.6320,37.305,0.0,117.621,0.000,7.0,spring,0,1,0
3,AS14.01,2014-03-23,6,0.0,1.0,2.141117,6033.096002,0.0,615.324,4495.049,...,900.8390,0.000,0.0,30.086,30.386,6.0,spring,0,1,0
4,AS14.01,2014-03-24,6,1.0,0.0,1.895065,10422.503001,0.0,816.953,6809.575,...,2644.4390,54.701,0.0,178.732,0.000,7.0,spring,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,AS14.33,2014-05-25,6,-1.0,0.0,3.713814,6659.054999,1.0,1282.030,1567.344,...,2868.6710,0.000,0.0,96.982,0.000,5.0,summer,1,0,0
1207,AS14.33,2014-05-26,5,-0.0,0.0,1.497619,5975.597999,3.0,846.084,1235.537,...,5441.2640,0.000,0.0,0.000,0.000,6.0,summer,1,0,0
1208,AS14.33,2014-05-27,6,-1.0,0.0,0.304902,3377.526001,2.0,365.069,2078.403,...,2010.3640,0.000,0.0,56.173,0.000,8.0,summer,1,0,0
1209,AS14.33,2014-05-28,8,0.0,1.0,2.479228,6241.620998,1.0,1751.404,1936.415,...,4487.4515,0.000,0.0,30.666,0.000,7.0,summer,1,0,0


In [16]:
target = []
for r in data['target_mood_plus1']:
    if r <= 5:
        target.append([1,0,0])
    if (r > 5) & (r <=7):
        target.append([0,1,0])
    if r > 7:
        target.append([0,0,1])

data['target'] = target
data = data.drop('time',axis = 1)
data = data.drop('season',axis =1)
data = data.drop('target_mood_plus1',axis = 1)

In [17]:
from sklearn.model_selection import train_test_split
X_train_total = []
X_test_total = []
Y_train_total = []
Y_test_total = []
ids = data['id'].unique()
labels = np.array(data['target'])
features= data.drop('target', axis = 1)
features = np.array(features)
Xtrain, Xtest, ytrain, ytest = train_test_split(features, labels, random_state = 42,stratify=labels)
X_train_total.append(Xtrain)
X_test_total.append(Xtest)
Y_train_total.append(ytrain)
Y_test_total.append(ytest)


In [18]:
# GET RID OF NESTED LIST
X_train_total = [item for l in X_train_total for item in l]
X_test_total = [item for l in X_test_total for item in l]
Y_train_total = [item for l in Y_train_total for item in l]
Y_test_total = [item for l in Y_test_total for item in l]

In [19]:
# DELETE USER ID
X_train_total = [item[1:] for item in X_train_total]
X_test_total = [item[1:] for item in X_test_total]

In [20]:
# # Using Skicit-learn to split data into training and testing sets
# from sklearn.model_selection import train_test_split
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state = 42, stratify=labels)

In [21]:
X_train_total

[array([6, -0.0, 1.0, 1.9007432950011167, 4224.460000038147, 0.0, 934.856,
        863.179, 359.939, 0.0, 0.0, 171.43, 48.70399999999999,
        2965.000000000002, 0.0, 0.0, 0.0, 0.0, 0, 1, 0], dtype=object),
 array([7, -1.0, 1.0, 3.2250461446744225, 1669.4170019626615, 1.0,
        274.7150000000001, 674.2520000000002, 293.6959999999998, 43.224,
        0.0, 0.0, 113.016, 0.0, 49.673, 0.0, 28.902, 0.0, 1, 0, 0],
       dtype=object),
 array([6, -0.0, 1.0, 3.030358047113633, 2049.531499385836, 0.0,
        234.99200000000002, 1542.119, 0.0, 0.0, 0.0, 0.0, 47.416,
        334.9980000000001, 0.0, 3.029, 0.0, 0.0, 1, 0, 0], dtype=object),
 array([7, 1.0, 1.0, 0.7579365079365087, 1661.3069999217978, 1.0, 312.297,
        1317.5410000000004, 0.0, 0.0, 0.0, 9.064, 8.031, 14.121, 0.0, 0.0,
        0.0, 0.0, 0, 1, 0], dtype=object),
 array([7, -0.0, 1.0, 6.07573282391933, 4566.025498032571, 1.0,
        700.6869999999997, 3225.006999999997, 1000.919, 0.0,
        179.90200000000004, 0.0, 181.

In [8]:
# accuracy_score(y_test, predictions.round())

In [9]:
# print(confusion_matrix(y_test, predictions.round()))

In [10]:
# import seaborn as sns
# # Get and reshape confusion matrix data
# matrix = confusion_matrix(y_test, predictions.round())
# matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# # Build the plot
# plt.figure(figsize=(16,7))
# sns.set(font_scale=1.4)
# sns.heatmap(matrix, annot=True, annot_kws={'size':10},
#             cmap=plt.cm.Greens, linewidths=0.2)

# # Add labels to the plot
# class_names = ['Negative', 'Neutral', 'Positive']
# tick_marks = np.arange(len(class_names))
# tick_marks2 = tick_marks + 0.5
# plt.xticks(tick_marks, class_names, rotation=25)
# plt.yticks(tick_marks2, class_names, rotation=0)
# plt.xlabel('Predicted label')
# plt.ylabel('True label')
# plt.title('Confusion Matrix for Random Forest Model')
# plt.show()

In [11]:
# print(classification_report(y_test, predictions.round()))

In [22]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [23]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier 

rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [24]:
rf_random.best_params_

{'n_estimators': 1200,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': True}

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [18, 20, 22],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2, 3],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [1100,1200, 1300]
}
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [26]:
grid_search.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  1.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [27]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 18,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 1100}

In [28]:
best_grid = grid_search.best_estimator_
best_grid

best_grid.fit(X_train_total, Y_train_total)
predictions = best_grid.predict(X_test_total)
# prob over the 3 classes
predictions = [np.argmax(p) for p in predictions]

In [29]:
for i in range (len(predictions)):
    if predictions[i] == 0:
        predictions[i] = [1,0,0]
    elif predictions[i] == 1:
        predictions[i] = [0,1,0]
    elif predictions[i] == 2:
        predictions[i] = [0,0,1]
#predictions

In [30]:
print(accuracy_score(Y_test_total, predictions))
print(classification_report(Y_test_total, predictions))
from sklearn.metrics import mean_squared_error

0.7755775577557755
              precision    recall  f1-score   support

           0       0.50      0.12      0.20         8
           1       0.78      0.97      0.86       217
           2       0.79      0.29      0.43        78

   micro avg       0.78      0.78      0.78       303
   macro avg       0.69      0.46      0.50       303
weighted avg       0.77      0.78      0.73       303
 samples avg       0.78      0.78      0.78       303



In [32]:
# Get numerical feature importances
features = data.drop('target', axis = 1)
feature_list = list(features.columns)

importances = list(best_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: activity             Importance: 0.1
Variable: circumplex.valence   Importance: 0.09
Variable: appCat.builtin       Importance: 0.09
Variable: appCat.office        Importance: 0.09
Variable: sms                  Importance: 0.08
Variable: appCat.communication Importance: 0.08
Variable: appCat.other         Importance: 0.08
Variable: id                   Importance: 0.06
Variable: appCat.game          Importance: 0.04
Variable: appCat.social        Importance: 0.04
Variable: appCat.unknown       Importance: 0.04
Variable: mood                 Importance: 0.03
Variable: circumplex.arousal   Importance: 0.03
Variable: screen               Importance: 0.03
Variable: appCat.entertainment Importance: 0.03
Variable: appCat.travel        Importance: 0.03
Variable: appCat.finance       Importance: 0.02
Variable: appCat.utilities     Importance: 0.02
Variable: appCat.weather       Importance: 0.01
Variable: summer               Importance: 0.01
Variable: spring               Importance